# ESO 137-001 - Image Reducer

The purpose of this notebook is to reduce the FLC files from Hubble by:

1. Aligning FLCs to the GAIA catalog
2. Drizzling Images together from a particular filter

## Imports

In [3]:
# Python Imports
from os import path
from glob import iglob

# Astropy Colab Imports
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import QTable
from drizzlepac.tweakreg import TweakReg
from drizzlepac.astrodrizzle import AstroDrizzle



The following task in the stsci.skypac package can be run with TEAL:
                                    skymatch                                    
The following tasks in the drizzlepac package can be run with TEAL:
    astrodrizzle       config_testbed      imagefindpars           mapreg       
       photeq            pixreplace           pixtopix            pixtosky      
  refimagefindpars       resetbits          runastrodriz          skytopix      
     tweakback            tweakreg           updatenpol


## Notebook Setup

In [4]:
# Data Directory
DATA_DIR = 'mastDownload/HST'

# FLC Glob Patter
FLC_CR_GLOB_PAT = path.join(DATA_DIR, '**/*_crclean_fl?.fits')

## Load the Data

In [ ]:
# Get the File Names and Sort them by filter
fileNameDict = {}
for fn in iglob(FLC_CR_GLOB_PAT):

    # Open the file to get the filter
    with fits.open(fn) as hduList:
        hdr = hduList[0].header  # Get the Header
        if 'FILTER' in hdr:      # If the FILTER keyword exists (WFC3)
            filt = hdr['FILTER']
        elif 'CLEAR' not in hdr['FILTER1']:  # If FILTER1 is not clear (ACS)
            filt = hdr['FILTER1']
        else:                                # Else FILTER2 must be the filter (ACS)
            filt = hdr['FILTER2']

    # Store the Name using the filter as the dict key
    # Start the Empty List if Key does not exist
    if filt not in fileNameDict:
        fileNameDict[filt] = []
    fileNameDict[filt].append(fn)

## Align Images to GAIA

### TweakReg Data Quality Flags

This cell defines the
[ACS DQ flags](https://www.stsci.edu/hst/instrumentation/acs/data-analysis/dq-flag-definitions),
[WFC3-UVIS DQ flags](https://hst-docs.stsci.edu/wfc3dhb/chapter-3-wfc3-data-calibration/3-2-uvis-data-calibration-steps#id-3.2UVISDataCalibrationSteps-3.2.3DataQualityArrayInitialization), and
[WFC3-IR DQ flags](https://hst-docs.stsci.edu/wfc3dhb/chapter-3-wfc3-data-calibration/3-3-ir-data-calibration-steps#id-3.3IRDataCalibrationSteps-3.3.1DataQualityInitialization)
we want to ignore in the TweakReg process.

In [6]:
# ACS DQ Bits
DQ_REP_PIX  = 2
DQ_BAD_DET  = 4
DQ_HOT_PIX  = 16
DQ_CR_PIX   = 4096+8192
DQ_GOOD_PIX_ACS = ~(DQ_REP_PIX + DQ_BAD_DET + DQ_HOT_PIX + DQ_CR_PIX)

# WFC3-UVIS DQ Bits
DQ_GOOD_PIX_WFC3_UVIS = ~(DQ_REP_PIX + DQ_BAD_DET + DQ_HOT_PIX + DQ_CR_PIX)

# WFC3-UVIS DQ Bits
DQ_DETECTOR_PROB = 4
DQ_GOOD_PIX_WFC3_IR = ~(DQ_REP_PIX + DQ_DETECTOR_PROB + DQ_HOT_PIX + 4096)

# DQ Dict
DQ_GOOD_DICT = {
    'F275W': DQ_GOOD_PIX_WFC3_UVIS,
    'F475W': DQ_GOOD_PIX_ACS,
    'F814W': DQ_GOOD_PIX_ACS,
    'F160W': DQ_GOOD_PIX_WFC3_IR
}

In [9]:
# Loop through Filters
for filter, fileList in fileNameDict.items():

    # Image Dep Search Parameters
    threshold  = 6.0 if filter != 'F275W' else 20.
    conv_width = 3.5 if filter != 'F160W' else 5.0

    # Image Find Parameters
    imagefindcfg = dict(
        # peakmax=900,
        threshold=threshold,
        conv_width=conv_width,
        dqbits=DQ_GOOD_DICT[filter]
    )

    # Align the Images to the GAIA data
    TweakReg(
        fileList,
        updatehdr=True,
        wcsname='GAIA',
        clean=True,
        configobj=None,
        refcat='../Data/GAIA/ESO137-001-GAIA-RefCatalog-icrs.txt',
        runfile=f'TweakReg-GAIA-{filter}.log',
        searchrad=0.35,
        minobj=25,
        tolerance=1,
        imagefindcfg=imagefindcfg,
        interactive=False
    )

Setting up logfile :  TweakReg-GAIA-F160W.log
TweakReg Version 3.7.1.1 started at: 10:34:56.878 (10/04/2025) 

Version Information
--------------------
Python Version 3.12.7 | packaged by conda-forge | (main, Oct  4 2024, 16:05:46) [GCC 13.3.0]
numpy Version -> 1.26.4 
astropy Version -> 7.0.0 
stwcs Version -> 1.7.3 
photutils Version -> 1.12.0 

Finding shifts for: 
    /home/wwaldron/Research/Galaxies/ESO/ESO-137-001/Images/mastDownload/HST/iblwa1n8q/iblwa1n8q_crclean_flt.fits
    /home/wwaldron/Research/Galaxies/ESO/ESO-137-001/Images/mastDownload/HST/iblwa1n7q/iblwa1n7q_crclean_flt.fits
    /home/wwaldron/Research/Galaxies/ESO/ESO-137-001/Images/mastDownload/HST/iblwa1naq/iblwa1naq_crclean_flt.fits

===  Source finding for image '/home/wwaldron/Research/Galaxies/ESO/ESO-137-001/Images/mastDownload/HST/iblwa1n8q/iblwa1n8q_crclean_flt.fits':
  #  Source finding for '/home/wwaldron/Research/Galaxies/ESO/ESO-137-001/Images/mastDownload/HST/iblwa1n8q/iblwa1n8q_crclean_flt.fits', EXT=('

In [10]:
%%bash

# Clean Up
mkdir -p logs/tweakreg
mv TweakReg-GAIA-*.log logs/tweakreg/
rm *.coo *.png

## Drizzle Images for CR Correction

Although there will be additional notes added later, it is worth noting that according to
[STScI](https://hst-docs.stsci.edu/drizzpac/chapter-6-reprocessing-with-the-drizzlepac-package/6-3-running-astrodrizzle#id-6.3RunningAstroDrizzle-SelectingtheOptimalScaleandPixfrac):

1. For sub-pixel dithered data, select an output scale that's smaller than the native scale.
It will even help in the cosmic ray rejection step.
1. A smaller final_pixfrac gives higher resolution and lower correlated noise, but also reduces
sensitivity to low-surface brightness features (though it is possible to convolve a high resolution
image later to go after low surface brightness features).
1. Keep the standard deviation of the weight map over the main part of the image to above ~0.3 of
the mean to insure that one does not lose significant signal-to-noise in ignoring the weight map in
final photometry.

To summarize the last step, a `final_scale`/`final_pixfrac` combo should be chosen such that,
for the weight image,
\begin{equation}
    \mathrm{std} \gtrsim 0.3 \, \mathrm{mean}
\end{equation}

### Note on Image Weights

AstroDrizzle can output [three different weight maps](https://drizzlepac.readthedocs.io/en/latest/drizzlepac_api/astrodrizzle.html):
exposure, "error" (appears to be legacy), and an inverse variance map. I have found that, in some cases, the exposure map
is more useful, and in other cases, the inverse variance map is more useful. Therefore, AstroDrizzle is run twice
below getting the two different types of weights. The files are then combined such that both weight types
are present in the final output file.

### AstroDrizzle Drizzle Data Quality Flags

* [ACS DQ flags](https://www.stsci.edu/hst/instrumentation/acs/data-analysis/dq-flag-definitions)
* [WFC3-UVIS DQ flags](https://hst-docs.stsci.edu/wfc3dhb/chapter-3-wfc3-data-calibration/3-2-uvis-data-calibration-steps#id-3.2UVISDataCalibrationSteps-3.2.3DataQualityArrayInitialization)
* [WFC3-IR DQ flags](https://hst-docs.stsci.edu/wfc3dhb/chapter-3-wfc3-data-calibration/3-3-ir-data-calibration-steps#id-3.3IRDataCalibrationSteps-3.3.1DataQualityInitialization)

In [11]:
# I know I should rewrite "constants", but here we are...

# ACS DQ Bits
DQ_GOOD_PIX_ACS = 64

# WFC3-UVIS DQ Bits
DQ_GOOD_PIX_WFC3_UVIS = 64

# WFC3-UVIS DQ Bits
DQ_GOOD_PIX_WFC3_IR = 64 + 512

# DQ Dict
DQ_GOOD_DICT = {
    'F275W': DQ_GOOD_PIX_WFC3_UVIS,
    'F475W': DQ_GOOD_PIX_ACS,
    'F814W': DQ_GOOD_PIX_ACS,
    'F160W': DQ_GOOD_PIX_WFC3_IR
}

### Drizzle F814W Image First for Reference

In [16]:
# Drizzle Constants
FINAL_SCALE = 0.03
FINAL_FRAC  = 0.7

In [22]:
# Drizzle Images
AstroDrizzle(
    fileNameDict['F814W'],
    output='ESO137-001-F814W',
    runfile='AstroDrizzle-F814W.log',
    wcskey='GAIA',
    context=False,
    configobj=None,
    num_cores=8,
    in_memory=False,
    build=True,
    restore=False,
    preserve=False,
    clean=True,
    skymethod='globalmin+match',
    driz_sep_scale=FINAL_SCALE,
    driz_sep_bits=DQ_GOOD_PIX_ACS,
    combine_type='minmed',
    driz_cr_corr=False,
    final_wht_type='IVM',
    final_pixfrac=FINAL_FRAC,
    final_bits=DQ_GOOD_PIX_ACS,
    final_wcs=True,
    final_rot=0,
    final_scale=FINAL_SCALE
)

# Drizzle Images with Exposure Map
AstroDrizzle(
    fileNameDict['F814W'],
    output='ESO137-001-F814W-exp',
    runfile='AstroDrizzle-F814W.log',
    wcskey='GAIA',
    context=False,
    configobj=None,
    num_cores=8,
    in_memory=False,
    build=True,
    restore=False,
    preserve=False,
    clean=True,
    skymethod='globalmin+match',
    driz_sep_scale=FINAL_SCALE,
    driz_sep_bits=DQ_GOOD_PIX_ACS,
    combine_type='minmed',
    driz_cr_corr=False,
    final_wht_type='EXP',
    final_pixfrac=FINAL_FRAC,
    final_bits=DQ_GOOD_PIX_ACS,
    final_wcs=True,
    final_rot=0,
    final_scale=FINAL_SCALE,
    final_refimage='ESO137-001-F814W_drc.fits'
)

Setting up logfile :  AstroDrizzle-F814W.log
AstroDrizzle log file: AstroDrizzle-F814W.log
AstroDrizzle Version 3.7.1.1 started at: 11:10:06.373 (10/04/2025)

==== Processing Step  Initialization  started at  11:10:06.375 (10/04/2025)



WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN' 'DEC--TAN' 
CRVAL : 243.3394109045248 -60.75681482054636 
CRPIX : 4086.774681738099 4033.160376420915 
CD1_1 CD1_2  : -8.333333333333335e-06 1.6122068945337695e-22 
CD2_1 CD2_2  : 2.8622624304831466e-22 8.333333333333335e-06 
NAXIS : 8172  8068
********************************************************************************
*
*  Estimated memory usage:  up to 3274 Mb.
*  Output image size:       8172 X 8068 pixels. 
*  Output image file:       ~ 754 Mb. 
*  Cores available:         4
*
********************************************************************************
==== Processing Step Initialization finished at 11:10:07.159 (10/04/2025)
==== Processing Step  Static Mask  started at  11:10:07.160 (10/04/2025)

==== Processing Step Static Mask finished at 11:10:07.759 (10/04/2025)
==== Processing Step  Subtract Sky  started at  11:10:07.760 (10/04/2025)

***** skymatch started on 2025-04-10 11:10:07.882438
      Version 1.0.10

'sky

In [ ]:
# Put Images Together
# Open First Image
with fits.open('ESO137-001-F814W_drc.fits', mode='update') as hduList:

    # Open Second Image
    with fits.open('ESO137-001-F814W-exp_drc.fits') as expList:

        # Update the Name of the IVM Map
        hduList['WHT'].header['EXTNAME'] = 'WHT-IVM'

        # Update the Name of the EXP Map
        expList['WHT'].header['EXTNAME'] = 'WHT-EXP'

        # Add the Exposure Map
        hduList.insert(3, expList['WHT-EXP'])

        # Save
        hduList.flush()

In [24]:
%%bash
# Remove Exposure Image
rm ESO137-001-F814W-exp_drc.fits

### Drizzle F475W Images

In [ ]:
# Drizzle Images
AstroDrizzle(
    fileNameDict['F475W'],
    output='ESO137-001-F475W',
    runfile='F475W-Astro.log',
    wcskey='GAIA',
    context=False,
    configobj=None,
    num_cores=8,
    in_memory=False,
    build=True,
    restore=False,
    preserve=False,
    clean=True,
    skymethod='globalmin+match',
    driz_sep_scale=0.03,
    driz_sep_bits=DQ_GOOD_PIX_ACS,
    combine_type='minmed',
    driz_cr_corr=False,
    final_wht_type='IVM',
    final_pixfrac=1,
    final_bits=DQ_GOOD_PIX_ACS,
    final_wcs=True,
    final_refimage='ESO137-001-F814W_drc.fits'
)

In [ ]:
# Clean Up
!mkdir astro
!mkdir -p ProcessedImages/HST/DrizzledImages
!mv *Astro.log astro/
!mv *drc.fits ProcessedImages/HST/DrizzledImages/